In [ ]:
# Берем двач очищенный от матов, размечаем по тематикам и затем токсиком и недопустимостью, также отдельно без тематик. размечаем руками, смотрим что по результатам

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
# df = pd.read_csv("for_final_test_no_obscene.csv")
df =pd.read_csv("for_final_test_no_obscene_autoalbel.csv")

In [3]:
df.head()

,text,inapp,toxic
0,МАМА МАМА Я В ИСТОРИИ)))И ТУТ КАПЧИ НЕТУ))),0.020365,0.028794
1,"Всем привет, с вами Игорь Линк",0.001812,0.886306
2,pokpok pok pokвся сутьсажа скрыл,0.577300,0.972156
3,"Наконец-то вы съехали. Рад за вас, ребят!",0.006442,0.547558
4,NUMBERЧтоб покемонобояре не утруждали себя кап...,0.465591,0.992220


In [11]:
from torch.utils.data import Dataset, DataLoader
class UnsafeDataset(torch.utils.data.Dataset):
    def __init__(self, text, labels=None):
        self.text = text

    def __getitem__(self, idx):
        item = self.text[idx]
        return item

    def __len__(self):
        return len(self.text)

In [12]:
inference_dataset = UnsafeDataset(df.text.tolist())
inference_dataloader = DataLoader(inference_dataset, batch_size = 32)

In [13]:
for b in inference_dataloader:
    pass
    break
# b

### Inappropriateness

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch


In [5]:
device = torch.device('cuda:5')

In [ ]:
model_name = 'Skoltech/russian-inappropriate-messages'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name);
model.to(device);

In [ ]:
res_inapp = []
for b in tqdm(inference_dataloader):
    with torch.no_grad():
        tokenized = tokenizer.batch_encode_plus(b,max_length = 512,
                                            pad_to_max_length=True,
                                            truncation=True,
                                            return_token_type_ids=False)
        tokens_ids,mask = torch.tensor(tokenized['input_ids']),torch.tensor(tokenized['attention_mask']) 
        tokens_ids = tokens_ids.to(device)
        mask = mask.to(device)
        model_output = model(tokens_ids,mask)
        soft = torch.softmax(model_output['logits'], dim = 1)
        soft = [el.cpu().item() for el in soft]
        res_inapp.extend(soft[:,1])
#     break

In [ ]:
df['inapp'] = res_inapp

In [ ]:
df.to_csv("for_final_test_no_obscene_autoalbel.csv",index = None)

### Toxicity

In [ ]:
model_name_toxic = 'sismetanin/rubert-toxic-pikabu-2ch'

tokenizer_toxic = BertTokenizer.from_pretrained(model_name_toxic)
model_toxic = BertForSequenceClassification.from_pretrained(model_name_toxic);


In [19]:
model_toxic.to(device);

In [22]:
res_toxic = []
for b in tqdm(inference_dataloader):
    with torch.no_grad():
        tokenized = tokenizer_toxic.batch_encode_plus(b,max_length = 512,
                                            pad_to_max_length=True,
                                            truncation=True,
                                            return_token_type_ids=False)
        tokens_ids,mask = torch.tensor(tokenized['input_ids']),torch.tensor(tokenized['attention_mask']) 
        tokens_ids = tokens_ids.to(device)
        mask = mask.to(device)
        model_output = model_toxic(tokens_ids,mask)
        soft = torch.softmax(model_output['logits'], dim = 1)
        soft = soft[:,1]
        soft = [el.cpu().item() for el in soft]
        res_toxic.extend(soft)
#     break

100%|██████████| 3736/3736 [24:15<00:00,  2.57it/s]


In [23]:
df['toxic'] = res_toxic

In [24]:
df.to_csv("for_final_test_no_obscene_autoalbel.csv",index = None)

### Sensitive topics

In [6]:
model_name = 'Skoltech/russian-sensitive-topics'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name);

In [15]:
model = model.to(device);

In [8]:
import json
with open("../sensitive_topics/id2topic.json") as f:
    target_vaiables_id2topic_dict = json.load( f)

In [9]:
def adjust_multilabel(y):
    y_adjusted = []
    for y_c in y:
        y_test_curr = [0]*19
        index = str(int(np.argmax(y_c)))
        y_c = target_vaiables_id2topic_dict[index]
    return y_c

In [34]:
res_topic = []
for b in tqdm(inference_dataloader):
    with torch.no_grad():
        tokenized = tokenizer.batch_encode_plus(b,max_length = 512,
                                            pad_to_max_length=True,
                                            truncation=True,
                                            return_token_type_ids=False)
        tokens_ids,mask = torch.tensor(tokenized['input_ids']),torch.tensor(tokenized['attention_mask']) 
        tokens_ids = tokens_ids.to(device)
        mask = mask.to(device)
        model_output = model(tokens_ids,mask)
        logits = model_output['logits'].cpu()
        preds = torch.argmax(logits,1).tolist()
        preds = [target_vaiables_id2topic_dict[str(p)] for p in preds]
        res_topic.extend(preds)
#         soft = torch.softmax(model_output['logits'], dim = 1)
#         soft = soft[:,1]
#         soft = [el.cpu().item() for el in soft]
#         res_toxic.extend(soft)
#     break

100%|██████████| 3736/3736 [24:12<00:00,  2.57it/s]


In [35]:
df['topic'] = res_topic

In [36]:
df.to_csv("for_final_test_no_obscene_autoalbel.csv",index = None)

# Select for labeling

In [21]:
from collections import Counter

In [10]:
df = pd.read_csv("for_final_test_no_obscene_autoalbel.csv")

In [11]:
df.head()

,text,inapp,toxic,topic
0,МАМА МАМА Я В ИСТОРИИ)))И ТУТ КАПЧИ НЕТУ))),0.020365,0.028794,none
1,"Всем привет, с вами Игорь Линк",0.001812,0.886306,none
2,pokpok pok pokвся сутьсажа скрыл,0.577300,0.972156,none
3,"Наконец-то вы съехали. Рад за вас, ребят!",0.006442,0.547558,none
4,NUMBERЧтоб покемонобояре не утруждали себя кап...,0.465591,0.992220,none


In [12]:
df = df.sample(frac=1, random_state = 1).reset_index(drop=True)

In [7]:
df.head()

,text,inapp,toxic,topic
0,"NUMBERНу вообще не у лягух, а у мелкого Ведра,...",0.006085,0.542380,none
1,NUMBERдержи без цензуры:url её когда она подкл...,0.442130,0.030260,none
2,NUMBERПетушок закукарекал Как там на личном? )),0.931578,0.400233,none
3,NUMBERНашел NUMBER гиг/NUMBERбит шина NUMBER н...,0.995333,0.991154,racism
4,"Ребят, а когда уже Witch появится в нормальном...",0.008427,0.035830,none


In [13]:
threshold = 0.75
df['toxic_inapp'] = df.apply(lambda x: (x['toxic']>threshold)&(x['inapp']>threshold), axis = 1)
df['toxic_app'] = df.apply(lambda x: (x['toxic']>threshold)&(x['inapp']<threshold), axis = 1)
df['notoxic_inapp'] = df.apply(lambda x: (x['toxic']<threshold)&(x['inapp']>threshold), axis = 1)
df['notoxic_app'] = df.apply(lambda x: (x['toxic']<threshold)&(x['inapp']<threshold), axis = 1)

In [16]:
df.head()

,text,inapp,toxic,topic,toxic_inapp,toxic_app,notoxic_inapp,notoxic_app
0,"NUMBERНу вообще не у лягух, а у мелкого Ведра,...",0.006085,0.542380,none,False,False,False,True
1,NUMBERдержи без цензуры:url её когда она подкл...,0.442130,0.030260,none,False,False,False,True
2,NUMBERПетушок закукарекал Как там на личном? )),0.931578,0.400233,none,False,False,True,False
3,NUMBERНашел NUMBER гиг/NUMBERбит шина NUMBER н...,0.995333,0.991154,racism,True,False,False,False
4,"Ребят, а когда уже Witch появится в нормальном...",0.008427,0.035830,none,False,False,False,True


In [15]:
df_no_topic = df[df['topic']=='none']
df_any_topic = df[df['topic']!='none']

In [37]:
Counter(df[df['toxic_app']]['toxic_app'])

Counter({True: 13498})

In [17]:
def get_tox_app_proportional_df(df_curr):
    collected_dfs = []
    for tox_inapp_rel_type in ['toxic_inapp','toxic_app','notoxic_inapp','notoxic_app']:
        df_curr_type = df_curr[df_curr[tox_inapp_rel_type]][:25]
        collected_dfs.append(df_curr_type)
    collected_dfs = pd.concat(collected_dfs)
    collected_dfs.drop_duplicates(subset=['text'],inplace = True)
    return collected_dfs

In [18]:
df_no_topic_for_label = get_tox_app_proportional_df(df_no_topic)

In [22]:
Counter(df_no_topic_for_label['toxic_inapp'])

Counter({True: 25, False: 75})

In [23]:
Counter(df_no_topic_for_label['notoxic_app'])

Counter({False: 75, True: 25})

In [24]:
len(df_no_topic_for_label)

100

In [32]:
# df_no_topic_for_label[df_no_topic_for_label['notoxic_inapp']==True]

In [33]:
# df_no_topic_for_label.sample(10)

In [25]:
df_any_topic_for_label = get_tox_app_proportional_df(df_any_topic)

In [28]:
Counter(df_any_topic_for_label['notoxic_app'])

Counter({False: 75, True: 25})

In [34]:
Counter(df_any_topic_for_label['toxic_app'])

Counter({False: 75, True: 25})

In [26]:
df_any_topic_for_label.to_csv("verification_any_topic.csv",index = None)

In [27]:
df_no_topic_for_label.to_csv("verification_no_topic.csv",index = None)